# 🔍 Multimodal RAG System: Pipeline Demo (без UI)

Мета — побудувати мультимодальну RAG-систему на базі статей з The Batch, що включає і текст, і зображення.


In [1]:
## бібліотеки
import requests
from bs4 import BeautifulSoup, SoupStrainer
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # для поділу тексту на шматки
from langchain.embeddings import HuggingFaceEmbeddings # для embeddings
from langchain_chroma import Chroma # для векторної бд
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from langchain.schema import Document

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Programming\anaconda\envs\rag_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. 📥 Завантаження статей та зображень

- Витягуємо текст і картинки (через web scraping, API або вручну).
- Зберігаємо сирі дані у `data/raw_articles/` та `data/images/`.

In [2]:
# url = "https://www.deeplearning.ai/the-batch/google-upgrades-its-ai-music-tools-for-professional-use/"
urls = ["https://www.deeplearning.ai/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom/","https://www.deeplearning.ai/the-batch/ai-co-scientist-an-agent-that-generates-research-hypotheses-aiding-drug-discovery/","https://www.deeplearning.ai/the-batch/ai-and-data-center-boom-challenges-big-techs-emissions-targets/"]


далі треба якось зпарсити все те тому використаю beatifulsoup 
pip install beautifulsoup4 lxml

In [3]:
contents = []
images = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    initial_images = [img.get('src') for img in soup.find_all('img') if img.get('src')]
    images.append([url for url in initial_images if 'gif' not in url.lower() and 'wordpress' not in url.lower() and 'svg+xml' not in url.lower() and 'batch-logo' not in url.lower()])
    elements = soup.select(".prose--styled") # вибираєсмо класи, записується в зворотньому порядку
    print(f"{url} - Знайдено contents {len(elements)} елементs")
    print(f"{url} - Знайдено imgs {len(images)} елементs")
    contents.append(elements[0].get_text(separator=' ')) # нормальний текст
contents
images    

https://www.deeplearning.ai/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom/ - Знайдено contents 1 елементs
https://www.deeplearning.ai/the-batch/the-international-energy-agency-examines-the-energy-costs-and-potential-savings-of-the-ai-boom/ - Знайдено imgs 1 елементs
https://www.deeplearning.ai/the-batch/ai-co-scientist-an-agent-that-generates-research-hypotheses-aiding-drug-discovery/ - Знайдено contents 1 елементs
https://www.deeplearning.ai/the-batch/ai-co-scientist-an-agent-that-generates-research-hypotheses-aiding-drug-discovery/ - Знайдено imgs 2 елементs
https://www.deeplearning.ai/the-batch/ai-and-data-center-boom-challenges-big-techs-emissions-targets/ - Знайдено contents 1 елементs
https://www.deeplearning.ai/the-batch/ai-and-data-center-boom-challenges-big-techs-emissions-targets/ - Знайдено imgs 3 елементs


[['/_next/image/?url=https%3A%2F%2Fcharonhub.deeplearning.ai%2Fcontent%2Fimages%2F2025%2F06%2Funnamed---2025-06-04T165349.311-1.png&w=3840&q=75'],
 ['/_next/image/?url=https%3A%2F%2Fcharonhub.deeplearning.ai%2Fcontent%2Fimages%2F2025%2F03%2Funnamed--65--1.png&w=3840&q=75'],
 ['/_next/image/?url=https%3A%2F%2Fcharonhub.deeplearning.ai%2Fcontent%2Fimages%2F2024%2F07%2Funnamed--70--1.jpg&w=3840&q=75']]

In [4]:
for content in contents:
    loader = WebBaseLoader(web_paths=urls)
    text_docs = loader.load()
text_docs[0].page_content = content

In [5]:
### перевірка (можна пропустити)
(len(text_docs[2].page_content))

4922

## 2. 🧹 Препроцесінг тексту та зображень (це взагалі треба?)

- Очищаємо HTML, витягуємо заголовки, основний текст.
- Перевірка розширень, нормалізація назв, зменшення зображень.


### текст

використовуємо langchain для поділу тексту на шматки (pip install langchain)

In [6]:
all_text_splits = []

for text_doc in text_docs:
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # chunk size (characters)
    chunk_overlap=100,  # chunk overlap (characters) (перекриття між суміжними шматками (50 символів (або слів) з кінця попереднього шматка повторюються на початку наступного)
    add_start_index=True,  # track index in original document
    )

    all_text_splits += text_splitter.split_documents(text_docs)

print(f"Split post into {len(all_text_splits)} sub-documents.")

Split post into 30 sub-documents.


pip install langchain-community (бо ембедінги не працюють)
pip install langchain-huggingface (проблеми пакетів)
pip install sentence-transformers (це все для лангчеін)

## 3. 🧠 Ембеддинг тексту

- Використовуємо SentenceTransformers або OpenAI для тексту.
- Зберігаємо ембединги + ідентифікатори статей.

In [7]:
#  Ініціалізація ембединг-моделі
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\1\AppData\Local\Temp\ipykernel_29604\1425009756.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [13]:
vector_1 = embeddings.embed_query(all_text_splits[0].page_content)
vector_2 = embeddings.embed_query(all_text_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(type(vector_1[:10]))

Generated vectors of length 768

<class 'list'>


## 4. 🖼️ Ембеддинг зображень

- Використовуємо CLIP, BLIP або ViT.
- Зберігаємо фічі зображень і прив’язуємо до статей.

pip install torch torchvision transformers Pillow requests

In [12]:
print(images[0][0])

/_next/image/?url=https%3A%2F%2Fcharonhub.deeplearning.ai%2Fcontent%2Fimages%2F2025%2F06%2Funnamed---2025-06-04T165349.311-1.png&w=3840&q=75


In [ ]:
# Завантажуємо модель і процесор
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name, use_fast=True)
img_embeddings = []
img_metadata = []

# Завантажуємо зображення з URL (або відкриваємо локальний файл)
for url in images:
    image = Image.open(requests.get('https://www.deeplearning.ai'+url[0], stream=True).raw).convert("RGB")# Якщо локальний файл: image = Image.open("path/to/image.jpg").convert("RGB")
    inputs = processor(images=image, return_tensors="pt")# Підготовка зображення для моделі

    # Отримуємо embedding
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)

    image_embedding = image_features / image_features.norm(p=2, dim=-1, keepdim=True) # Нормалізуємо вектор (добре для пошуку за косинусною схожістю)


    # Перетворюємо у numpy-масив (зручний формат для збереження)
    # embedding_vector = image_embedding.cpu().numpy()[0]
    print(type(image_features.squeeze().cpu().tolist()))
    img_embeddings.append(image_features.squeeze().cpu().tolist())
    img_metadata.append({"url": url[0], "id":None})
    print("Розмір embedding:", len())


<class 'list'>
Розмір embedding: 512
<class 'list'>
Розмір embedding: 512
<class 'list'>
Розмір embedding: 512


In [29]:
img_docs = []
for emb, meta in zip(img_embeddings, img_metadata):
    # Створюємо Document з порожнім текстом, щоб зберегти embedding та метадані
    img_docs.append(Document(page_content="", metadata=meta))


## 5. 🗃️ Створення мультимодального індексу

- Об’єднуємо текстові + візуальні ембединги.
- Індексування за допомогою FAISS / Chroma / Milvus.

pip install chromadb (я хз чи це треба)
pip install langchain-chroma


In [9]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="chroma_langchain_db/",  # Where to save data locally, remove if not necessary
)


In [10]:
vector_store.add_documents(documents=all_text_splits)

['f38d6c4e-4b61-4758-aeb3-708c3a853492',
 '1657efb0-1bff-4f1f-b4b4-643c84027b83',
 '1d633b72-6835-4e4a-a7a1-644f250bc91b',
 'e782133f-6ce8-413c-8673-0f9b20b245d9',
 '960af1bb-c956-408d-818e-dec0006f5ba8',
 'fb22e543-8879-4418-8fe9-2a17e407f84f',
 '549eed4e-c153-4118-8c99-888ab41ce0c8',
 'f2755d89-2bff-4a71-aaf3-e830b2b88fe5',
 '0e7d2f4f-42b8-4b35-829d-20bf61333fa6',
 'e5d791c6-9bdf-4499-aaea-1890fbdcbc5f',
 '2e632670-a103-4552-8de9-b3e80f2a67c3',
 '87d3997f-0112-41ee-9613-aa580fc39a31',
 '16bdba5d-e2f8-415e-9320-1318e47d049a',
 '906082bc-09a8-4952-808b-75f99002dd70',
 'ff556279-cb72-48d7-a095-afd634c8e637',
 '78205af8-fea1-4126-8fe3-e9177aae7267',
 'f3556cd9-be36-44a0-82b9-d4a254d738d7',
 '9577d74f-0ca3-43f8-8564-d305ad1483a3',
 'f5f9ef2a-f2ea-497f-a1b4-e25d316c1b9e',
 '9c1da78a-6989-4a11-b082-90c054c2070d',
 '28d47d7f-6b56-4af7-b2bd-6b806d94e964',
 'e0dcec7a-5568-4656-8c8a-a10ffae54045',
 'cb9aaec7-b883-4835-b5b1-f9473183ef8e',
 'a0f3aaed-157f-4d22-89aa-8291055ea46c',
 '359673aa-8d89-

In [30]:
vector_store.add_documents(img_docs, embeddings=img_embeddings)

['66f5694e-4eba-4e3a-8f45-dc274afeb9a6',
 'd0c13dab-7be0-4b3c-951e-6d2b6237b141',
 'd199bad1-ce7d-46f5-abbf-dd0c5ae53794']

## 6. ❓Запит і Ретрівал

- Користувач формулює запит (наприклад: “Що нового в архітектурах NVIDIA?”).
- Ембеддинг запиту.
- Пошук у мультимодальному індексі.
- Вивід: текст статті + пов’язане зображення.

In [12]:
results = vector_store.similarity_search(
    "How is AI growth impacting tech companies' carbon goals and data center emissions?"
)

print((results[0]))

page_content='AI and Data Center Boom Challenges Big Tech's Emissions Targets✨ New course! Enroll in Building with Llama 4Explore CoursesAI NewsletterThe BatchAndrew's LetterData PointsML ResearchBlogCommunityForumEventsAmbassadorsAmbassador SpotlightResourcesCompanyAboutCareersContactStart LearningWeekly IssuesAndrew's LettersData PointsML ResearchBusinessScienceCultureHardwareAI CareersAboutSubscribeThe BatchTech & SocietyArticleAI’s Path to Zero Emissions Is Cloudy AI and data center boom challenges big tech's emissions targetsTech & SocietyScience#Import 2025-06-13 02:32PublishedJul 10, 2024Reading time3 min readShareThe boom in AI is jeopardizing big tech’s efforts to reach its targets for emissions of greenhouse gasses.What’s new: Google’s annual environmental report shows that the company’s total carbon dioxide emissions rose nearly 50 percent between 2019 and 2023 to 14.3 million tons. Google attributes the rise to its efforts to satisfy rising demand for AI. How it works: Goog

## 7. ✅ Підсумок і оцінка

- Якість відповідей
- Наскільки релевантні результати